In [3]:
import os
import numpy as np
import json
import pandas as pd
import datetime as DT

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AX4XfWjfwQSlr0NRGGr2MfOuYbS-7K47SfYKUtuvRX2mlGX6ySB7V604qPU
Mounted at /content/drive


# Get required data
From the github repository

In [ ]:
# Clone the entire repo.
!git clone -l -s https://github.com/XLipcak/rev-rec.git cloned-repo

Cloning into 'cloned-repo'...
remote: Enumerating objects: 1170, done.
remote: Total 1170 (delta 0), reused 0 (delta 0), pack-reused 1170
Receiving objects: 100% (1170/1170), 45.11 MiB | 15.19 MiB/s, done.
Resolving deltas: 100% (503/503), done.
Checking out files: 100% (88/88), done.


In [ ]:
%ls

cloned-repo/  drive/  sample_data/


In [ ]:
# Save OpenStack dataset
import shutil
src = "/content/cloned-repo/rev-rec-data/openstack.json"
dst = "/content/drive/MyDrive/Colab Notebooks/My Project/openstack.json"
shutil.move(src, dst)

'/content/drive/MyDrive/Colab Notebooks/My Project/openstack.json'

# Read json file

In [18]:
f = open('/content/drive/MyDrive/Colab Notebooks/My Project/openstack.json',)
data = json.load(f)  # returns json object as a dictionary
for i in range(5):   # 5 first elements of the json list
    print(data[i])
f.close()

{'subProject': 'openstack/tempest', 'changeId': 'I9e7b9a2491d6a3f01d71551e4cadcccf154d992d', 'changeNumber': 6139, 'timestamp': 1333428006000, 'reviewers': [{'accountId': '97', 'email': 'daryl.walleck@rackspace.com', 'name': 'Daryl Walleck', 'avatar': None}, {'accountId': '7', 'email': 'jaypipes@gmail.com', 'name': 'Jay Pipes', 'avatar': None}], 'owner': {'accountId': '2238', 'email': 'rajalakshmi.girish@hpe.com', 'name': 'rajalakshmi-ganesan', 'avatar': None}, 'filePaths': [{'location': 'tempest/tests/test_volumes_get.py'}]}
{'subProject': 'openstack/tempest', 'changeId': 'I10e7116570f922ec87e23b1f880cd4b1c08c3088', 'changeNumber': 7438, 'timestamp': 1337072165000, 'reviewers': [{'accountId': '97', 'email': 'daryl.walleck@rackspace.com', 'name': 'Daryl Walleck', 'avatar': None}, {'accountId': '7', 'email': 'jaypipes@gmail.com', 'name': 'Jay Pipes', 'avatar': None}], 'owner': {'accountId': '4120', 'email': 'rohit.karajgi@ril.com', 'name': 'Rohit Karajgi', 'avatar': None}, 'filePaths': 

## Hands on with given data

In [ ]:
type(data)

list

In [ ]:
len(data)

6545

In [ ]:
# get an element of the list of data
data[0]

{'changeId': 'I9e7b9a2491d6a3f01d71551e4cadcccf154d992d',
 'changeNumber': 6139,
 'filePaths': [{'location': 'tempest/tests/test_volumes_get.py'}],
 'owner': {'accountId': '2238',
  'avatar': None,
  'email': 'rajalakshmi.girish@hpe.com',
  'name': 'rajalakshmi-ganesan'},
 'reviewers': [{'accountId': '97',
   'avatar': None,
   'email': 'daryl.walleck@rackspace.com',
   'name': 'Daryl Walleck'},
  {'accountId': '7',
   'avatar': None,
   'email': 'jaypipes@gmail.com',
   'name': 'Jay Pipes'}],
 'subProject': 'openstack/tempest',
 'timestamp': 1333428006000}

In [ ]:
# read a single feature of an element
data[0]['reviewers']

[{'accountId': '97',
  'avatar': None,
  'email': 'daryl.walleck@rackspace.com',
  'name': 'Daryl Walleck'},
 {'accountId': '7',
  'avatar': None,
  'email': 'jaypipes@gmail.com',
  'name': 'Jay Pipes'}]

## Cleaning data

In [19]:
# Extract and save desired features from each pull request. The new desired data is new_data
new_data = []

for i in range(0, len(data)):
  elem_dict = {"changeId":data[i]['changeId'], "reviewers":data[i]['reviewers'], "owner":data[i]['owner'], "filePaths":data[i]['filePaths'], "time":data[i]['timestamp']}
  new_data.append(elem_dict)
  # print(new_data[i])

print(len(new_data))

6545


In [20]:
new_data[0]

{'changeId': 'I9e7b9a2491d6a3f01d71551e4cadcccf154d992d',
 'filePaths': [{'location': 'tempest/tests/test_volumes_get.py'}],
 'owner': {'accountId': '2238',
  'avatar': None,
  'email': 'rajalakshmi.girish@hpe.com',
  'name': 'rajalakshmi-ganesan'},
 'reviewers': [{'accountId': '97',
   'avatar': None,
   'email': 'daryl.walleck@rackspace.com',
   'name': 'Daryl Walleck'},
  {'accountId': '7',
   'avatar': None,
   'email': 'jaypipes@gmail.com',
   'name': 'Jay Pipes'}],
 'time': 1333428006000}

In [21]:
# Remove remaining undesired features. The final desired information is saved in final_data
final_data = []

for i in range(0, len(new_data)):
  elem_dict = {"time":new_data[i]['time'], "changeId":new_data[i]['changeId'], "ownerId": new_data[i]['owner']['accountId'], "reviewersId":[], "filePaths":[]}

  for j in range(0, len(new_data[i]['reviewers'])):
    elem_dict["reviewersId"].append(new_data[i]['reviewers'][j]['accountId'])

  for j in range(0, len(new_data[i]['filePaths'])):
    elem_dict["filePaths"].append(new_data[i]['filePaths'][j]['location'])
  
  final_data.append(elem_dict)
  # print(final_data[i])

print(len(final_data))

6545


In [22]:
final_data[0]

{'changeId': 'I9e7b9a2491d6a3f01d71551e4cadcccf154d992d',
 'filePaths': ['tempest/tests/test_volumes_get.py'],
 'ownerId': '2238',
 'reviewersId': ['97', '7'],
 'time': 1333428006000}

In [24]:
# change timestamp value from int to date format
from datetime import datetime

for pr in final_data:
  timestamp = int(pr['time']/1000)  # removing the 3 last zero digits 
  dt_obj = datetime.fromtimestamp(timestamp) 
  pr['time'] = dt_obj

In [25]:
final_data[0]

{'changeId': 'I9e7b9a2491d6a3f01d71551e4cadcccf154d992d',
 'filePaths': ['tempest/tests/test_volumes_get.py'],
 'ownerId': '2238',
 'reviewersId': ['97', '7'],
 'time': datetime.datetime(2012, 4, 3, 4, 40, 6)}

In [26]:
# sort data by time
sorted_data = sorted(final_data, key=lambda d: d['time'])

# Get the Time of available data from selected project
print("Start date:", sorted_data[0]['time'])
print("End date:", sorted_data[-1]['time'])

Start date: 2011-07-18 15:43:34
End date: 2012-05-30 21:39:57


The openstack data that we use in here, containes about 10 months information of openstack project (2011-2012)

In [27]:
type(final_data)

list

In [31]:
# Convert list to json
# It should be a string before write it to file
# "default" is added to dumps to make anything not serializable for JSON to string format (like datatime) 
json_string = json.dumps(sorted_data, default=str) 

# Save OpenStack new dataset
file = open('openstack_final.json', 'w')
file.write(json_string)
file.close()

In [32]:
# Move OpenStack dataset to proper location
import shutil

src = "./openstack_final.json"
dst = "/content/drive/MyDrive/Colab Notebooks/My Project/openstack_final.json"
shutil.move(src, dst)

'/content/drive/MyDrive/Colab Notebooks/My Project/openstack_final.json'

# Approach

Should run the code each time from this part

## Load final data and split

In [96]:
# Load final data from drive
f = open('/content/drive/MyDrive/Colab Notebooks/My Project/openstack_final.json',)
data = json.load(f)  # returns json object as a dictionary
for i in range(5):   # 5 first elements of the json list
    print(data[i])
f.close()

{'time': '2011-07-18 15:43:34', 'changeId': 'Ibb2f047c9c45d011361d253d4444b2fd1ebd3612', 'ownerId': '1', 'reviewersId': ['2'], 'filePaths': ['modules/ssh/files/sshd_config', 'manifests/server.pp', 'modules/ssh/manifests/init.pp']}
{'time': '2011-07-18 16:56:22', 'changeId': 'Iea32ad3aedbee996db53655842061d14c9e3d876', 'ownerId': '1', 'reviewersId': ['1'], 'filePaths': ['modules/jenkins_slave/manifests/jenkinsuser.pp', 'modules/jenkins_slave/files/known_hosts']}
{'time': '2011-07-18 17:14:05', 'changeId': 'Ib6af08a6af71a6e5853d958a12f63c381eafa039', 'ownerId': '1', 'reviewersId': ['1'], 'filePaths': ['modules/jenkins_slave/manifests/jenkinsuser.pp']}
{'time': '2011-07-20 14:37:53', 'changeId': 'Ide7975c522b6b31a23af67b135135c07d75d9bde', 'ownerId': '2', 'reviewersId': ['1'], 'filePaths': ['modules/jenkins_slave/files/pubring.gpg', 'modules/jenkins_slave/manifests/jenkinsuser.pp']}
{'time': '2011-07-20 15:12:47', 'changeId': 'I28119ac1735127812866a4b0519368a829e2e666', 'ownerId': '2', 'r

In [97]:
# convert times to datetime format
from dateutil import parser

for pr in data:
  pr['time'] = parser.parse(pr['time'])

data[0]

{'changeId': 'Ibb2f047c9c45d011361d253d4444b2fd1ebd3612',
 'filePaths': ['modules/ssh/files/sshd_config',
  'manifests/server.pp',
  'modules/ssh/manifests/init.pp'],
 'ownerId': '1',
 'reviewersId': ['2'],
 'time': datetime.datetime(2011, 7, 18, 15, 43, 34)}

In [98]:
# Split data for train/test sets
init_train = data[0:5900]     # 90% of data is for initial train set (5900 PR)
total_test = data[5900:6545]  # 10% of data will be tested in the whole examination (645 PR)

## Main components of the approach
Extract 3 interaction metrices

### Reviewer's **expertise** model

#### step by step

In [42]:
all_files = set()   # make a "set" of all file paths. we selected set to prevent collecting repeated files
for pr in train:
  for f in pr['filePaths']:
    all_files.add(f)

len(all_files)

11072

In [53]:
all_reviewers = set()   # make a set of all reviewers
for pr in train:
  for r in pr['reviewersId']:
    all_reviewers.add(r)

len(all_reviewers)

78

In [54]:
# create a dataframe that its first column is all existing files
df = pd.DataFrame(all_files)
df.head()

,0
0,horizon/horizon/dashboards/syspanel/templates/...
1,doc/src/docbkx/openstack-compute-admin/target/...
2,bin/clear_rabbit_queues
3,doc/target/docbkx/webhelp/trunk/openstack-obje...
4,tools/build_lxc_multi.sh


In [55]:
df = df.rename(columns={0:'file'})
df.head()

,file
0,horizon/horizon/dashboards/syspanel/templates/...
1,doc/src/docbkx/openstack-compute-admin/target/...
2,bin/clear_rabbit_queues
3,doc/target/docbkx/webhelp/trunk/openstack-obje...
4,tools/build_lxc_multi.sh


In [56]:
# add a column of zeros for each existing reviewer to dataframe
for r in all_reviewers:
  df[r] = 0

df.head()

,file,161,1228,1030,100,261,1132,209,1998,4,67,642,1205,1179,970,494,2170,191,24,490,2472,1549,5,1155,308,679,330,2243,360,357,1816,70,236,1603,34,1009,233,845,693,2828,512,979,681,1216,1689,452,1247,1,335,518,112,2,7,616,1561,917,475,6,1456,2458,447,704,97,2622,2325,2230,543,964,385,927,2035,688,706,995,2284,8,321,1192,941
0,horizon/horizon/dashboards/syspanel/templates/...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,doc/src/docbkx/openstack-compute-admin/target/...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,bin/clear_rabbit_queues,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,doc/target/docbkx/webhelp/trunk/openstack-obje...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,tools/build_lxc_multi.sh,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [57]:
# fill the values of dataframe by checking PRs one by one
for pr in train:
  for f in pr['filePaths']:
    for r in pr['reviewersId']:
      df.loc[(df['file'] == f), r] += 1

df.head()

,file,161,1228,1030,100,261,1132,209,1998,4,67,642,1205,1179,970,494,2170,191,24,490,2472,1549,5,1155,308,679,330,2243,360,357,1816,70,236,1603,34,1009,233,845,693,2828,512,979,681,1216,1689,452,1247,1,335,518,112,2,7,616,1561,917,475,6,1456,2458,447,704,97,2622,2325,2230,543,964,385,927,2035,688,706,995,2284,8,321,1192,941
0,horizon/horizon/dashboards/syspanel/templates/...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,doc/src/docbkx/openstack-compute-admin/target/...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0
2,bin/clear_rabbit_queues,0,0,1,0,0,1,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,doc/target/docbkx/webhelp/trunk/openstack-obje...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0
4,tools/build_lxc_multi.sh,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [58]:
df = df.set_index('file')
df.head()

,161,1228,1030,100,261,1132,209,1998,4,67,642,1205,1179,970,494,2170,191,24,490,2472,1549,5,1155,308,679,330,2243,360,357,1816,70,236,1603,34,1009,233,845,693,2828,512,979,681,1216,1689,452,1247,1,335,518,112,2,7,616,1561,917,475,6,1456,2458,447,704,97,2622,2325,2230,543,964,385,927,2035,688,706,995,2284,8,321,1192,941
file,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
horizon/horizon/dashboards/syspanel/templates/syspanel/instances/_reboot.html,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
doc/src/docbkx/openstack-compute-admin/target/docbkx/webhelp/os-compute-adminguide/common/images/showHideTreeIcons.png,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0
bin/clear_rabbit_queues,0,0,1,0,0,1,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
doc/target/docbkx/webhelp/trunk/openstack-object-storage/admin/content/partition-assignment-list.html,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0
tools/build_lxc_multi.sh,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [59]:
# convert the dataframe into matrix
FR = df.to_numpy()
FR.shape

(11072, 78)

#### All together
as 'cal_expertise' function

In [60]:
# Function to extract 'Expertise' metrix
# Returns P*M matrix named FR
def cal_expertise(train, test):

  # create a "set" of all file paths. we selected set to prevent collecting repeated files
  all_files = set()   
  for pr in train:
    for f in pr['filePaths']:
      all_files.add(f)
  print("P is:", len(all_files))

  # create a set of all reviewers
  all_reviewers = set()   
  for pr in train:
    for r in pr['reviewersId']:
      all_reviewers.add(r)
  print("M is:", len(all_reviewers))

  # create a dataframe that its first column is all existing files
  df = pd.DataFrame(all_files)  
  df = df.rename(columns={0:'file'})  # set a name for this column

  # add a column of zeros for each existing reviewer to dataframe
  for r in all_reviewers:
    df[r] = 0

  # fill the cells of the dataframe by checking PRs one by one
  for pr in train:
    for f in pr['filePaths']:
      for r in pr['reviewersId']:
        df.loc[(df['file'] == f), r] += 1

  # convert the dataframe into matrix
  df = df.set_index('file')
  FR = df.to_numpy()
  print("Shape of FR matrix is:", FR.shape)
  return FR

### Reviewer‑developer **collaboration** model

#### step by step

In [61]:
all_devs = set()   # make a "set" of all developers
for pr in train:
  for d in pr['ownerId']:
    all_devs.add(d)

len(all_devs)

10

In [62]:
len(all_reviewers)

78

In [63]:
# create a dataframe that its first column is all existing developers
df = pd.DataFrame(all_devs)
df = df.rename(columns={0:'developer'})
df

,developer
0,9
1,8
2,0
3,3
4,2
5,4
6,5
7,1
8,7
9,6


In [64]:
# add a column of zeros for each existing reviewer to dataframe
for r in all_reviewers:
  df[r] = 0

df.head()

,developer,161,1228,1030,100,261,1132,209,1998,4,67,642,1205,1179,970,494,2170,191,24,490,2472,1549,5,1155,308,679,330,2243,360,357,1816,70,236,1603,34,1009,233,845,693,2828,512,979,681,1216,1689,452,1247,1,335,518,112,2,7,616,1561,917,475,6,1456,2458,447,704,97,2622,2325,2230,543,964,385,927,2035,688,706,995,2284,8,321,1192,941
0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [65]:
# fill the values of dataframe by checking PRs one by one
for pr in train:
  for r in pr['reviewersId']:
    df.loc[(df['developer'] == pr['ownerId']), r] += len(pr['filePaths'])

df.head()

,developer,161,1228,1030,100,261,1132,209,1998,4,67,642,1205,1179,970,494,2170,191,24,490,2472,1549,5,1155,308,679,330,2243,360,357,1816,70,236,1603,34,1009,233,845,693,2828,512,979,681,1216,1689,452,1247,1,335,518,112,2,7,616,1561,917,475,6,1456,2458,447,704,97,2622,2325,2230,543,964,385,927,2035,688,706,995,2284,8,321,1192,941
0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,8,0,0,0,0,0,0,0,0,504,0,0,0,0,0,0,0,0,0,0,0,0,193,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,0,0,0,0,0,0,0,127,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,76,0,6,0,0,0,59,0,0,0,0,0,0,0,0,0,0,0,0,0,0,78,0,0,0,19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,59,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,58,0,0
4,2,0,0,0,41,21,29,2,0,12,31,1,3,0,3,52,0,0,0,0,0,0,72,0,0,35,2,0,1,3,7,0,0,0,0,0,40,0,0,0,0,0,0,0,0,0,25,211,0,5,7,166,25,0,0,0,0,8,4,0,56,4,0,0,49,0,0,0,48,0,1,0,4,0,4,0,0,0,0


In [66]:
df = df.set_index('developer')
# convert the dataframe into matrix
DR = df.to_numpy()
DR.shape

(10, 78)

#### All together
as 'cal_collaboration' function

In [67]:
# Function to extract 'Collaboration' metrix
# Returns N*M matrix named DR
def cal_collaboration(train, test):

  # create a "set" of all developers
  all_devs = set()  
  for pr in train:
    for d in pr['ownerId']:
      all_devs.add(d)
  print("N is:", len(all_devs))

  # There is a set of all reviewers
  print("M is:", len(all_reviewers))

  # create a dataframe that its first column is all existing developers
  df = pd.DataFrame(all_devs)
  df = df.rename(columns={0:'developer'})

  # add a column of zeros for each existing reviewer to dataframe
  for r in all_reviewers:
    df[r] = 0
  
  # fill the values of dataframe by checking PRs one by one
  for pr in train:
    for r in pr['reviewersId']:
      df.loc[(df['developer'] == pr['ownerId']), r] += len(pr['filePaths'])

  # convert the dataframe into matrix
  df = df.set_index('developer')
  DR = df.to_numpy()
  print("Shape of DR matrix is:", DR.shape)
  return DR

### **Availability** model

#### step by step

In [81]:
len(all_reviewers)

78

In [82]:
test['time']

datetime.datetime(2012, 5, 3, 21, 32, 38)

In [83]:
import datetime as DT

# seven day before the time of our test PR
week_ago = test['time'] - DT.timedelta(days=7)
week_ago

datetime.datetime(2012, 4, 26, 21, 32, 38)

In [90]:
# Extract a list of PRs that are between time of seven days ago up to the time of test PR
week_pr = []

for pr in train:
  if pr['time'] > week_ago and pr['time'] < test['time']:
    week_pr.append(pr)

len(week_pr)

214

In [93]:
# Extract the availability matrix named 'A' that is 1*M
A = []

for r in all_reviewers:
  A.append(0)
  for pr in week_pr:
    if r in pr['reviewersId']:
      A[-1] += len(pr['filePaths'])

len(A)

78

#### All together
as 'cal_availability' function

In [95]:
# Function to extract 'Availability' vector
# Returns 1*M matrix named A 
def cal_availability(train, test):

  # seven day before the time of our test PR
  week_ago = test['time'] - DT.timedelta(days=7)
  
  # Extract a list of PRs that are between time of seven days ago up to the time of test PR
  week_pr = []
  for pr in train:
    if pr['time'] > week_ago and pr['time'] < test['time']:
      week_pr.append(pr)
  print("Num of PRs in last 7 days:", len(week_pr))

  # Extract the availability matrix named 'A' that is 1*M
  A = []
  for r in all_reviewers:
    A.append(0)
    for pr in week_pr:
      if r in pr['reviewersId']:
        A[-1] += len(pr['filePaths'])
  
  print("Size of A matrix is:", len(A))
  return A

## Test

In [99]:
for test_num in range(0, 1):  # number of examination can be 0 to 644
  # creating test set
  test = total_test[test_num]   # one PR is test in each examination
  # creating train set
  if test_num == 0:
    train = init_train
  else:
    train.append(total_test[test_num-1])
  
  # Do other calculations
  # extract 3 metrices
  FR = cal_expertise(train, test)
  print(FR)
  print("------------------------------")
  DR = cal_collaboration(train, test)
  print(DR)
  print("------------------------------")
  A = cal_availability(train, test)
  print(A)
  # do genetic
  # ...



P is: 11072
M is: 78
Shape of FR matrix is: (11072, 78)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
------------------------------
N is: 10
M is: 78
Shape of DR matrix is: (10, 78)
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0 504   0   0   0   0   0   0   0   0   0
    0   0   0 193   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0  29   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0  13   0   0   0   0   0
    0   0 127   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0  

In [ ]:
train[0:5]

[{'changeId': 'I9e7b9a2491d6a3f01d71551e4cadcccf154d992d',
  'filePaths': ['tempest/tests/test_volumes_get.py'],
  'ownerId': '2238',
  'reviewersId': ['97', '7']},
 {'changeId': 'I10e7116570f922ec87e23b1f880cd4b1c08c3088',
  'filePaths': ['tempest/tests/identity/test_roles.py',
   'tempest/services/identity/json/admin_client.py',
   'tempest/tests/identity/base_admin_test.py'],
  'ownerId': '4120',
  'reviewersId': ['97', '7']},
 {'changeId': 'I1d0c395cf679e07e304dd878d97734307b685f4c',
  'filePaths': ['.gitignore'],
  'ownerId': '1849',
  'reviewersId': ['679', '1561']},
 {'changeId': 'Ia151f265bf62e6704de6cc2f2c01963be9e2dd69',
  'filePaths': ['tools/hacking.py'],
  'ownerId': '1849',
  'reviewersId': ['1561', '357']},
 {'changeId': 'Ie4cfec6f89d6b37554b5345cde92f982397915e2',
  'filePaths': ['tox.ini'],
  'ownerId': '2',
  'reviewersId': ['1561', '357']}]

In [ ]:
test

{'changeId': 'I0064ddec13b090fe8a6b7c0dde2feda29d277828',
 'filePaths': ['manifests/site.pp'],
 'ownerId': '1',
 'reviewersId': ['2']}